# Importar dados

In [67]:
import numpy as np
# Carregar os dados
feature = np.load('data/feature.npy')
target = np.load('data/target.npy')

# Infraestrutura dos dados (dataloader)

## Normalizando

In [68]:
from sklearn.preprocessing import MinMaxScaler

In [69]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Normalização das features
X_scaled = scaler_X.fit_transform(feature.reshape(-1, feature.shape[-1])).reshape(feature.shape)
y_scaled = scaler_y.fit_transform(target)

## Criando um dataSet e utilizando o dataloader

In [70]:
from torch.utils.data import DataLoader, Dataset

In [71]:
class CustomDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label

    def __len__(self):
        return len(self.feature)

    def __getitem__(self, index):
        return self.feature[index], self.label[index]


# Modelo

In [72]:
from torch import nn

In [73]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.linear(lstm_out[:, -1, :])

# Funções

## Treino

In [74]:
def train(modelo, dataloader, funcao_de_custo, otimizador, device):
    modelo.train()
    erro_acumulativo = 0.0
    for _, (data, target) in enumerate(dataloader):
        target, datas = target.float().to(device), data.float().to(device)
        pred = modelo(datas)
        perda = funcao_de_custo(pred, target)
        otimizador.zero_grad()
        perda.backward()
        otimizador.step()
        erro_acumulativo += perda.item()
    return erro_acumulativo / len(dataloader)

## Teste

In [75]:
import torch

In [76]:
def test(modelo, dataloader, funcao_de_custo, device):
    modelo.eval()
    total_loss = 0.0
    with torch.no_grad():
        for _, (data, target) in enumerate(dataloader):
            target, datas = target.float().to(device), data.float().to(device)
            pred = modelo(datas)
            loss = funcao_de_custo(pred, target)
            total_loss += loss.item()
    return total_loss / len(dataloader)

## Criação dos Graficos

In [77]:
import matplotlib.pyplot as plt

In [78]:
def plot_results(fold_results, val_losses, train_losses, all_predictions, all_real_values):
    # 1. Plotar a perda de validação por fold
    plt.figure(figsize=(10, 6))
        
    plt.plot(val_losses, label=f'Fold')
    
    plt.title("Perda de Validação por Fold")
    plt.xlabel("Folds")
    plt.ylabel("Perda de Validação (Loss)")
    plt.legend(title="Folds")
    plt.show()

    plt.figure(figsize=(10, 6))
    for loss in range(len(train_losses)):
        plt.plot(train_losses[loss], label=f'Perda de Treinamento (Fold {loss+1})', marker='o', linestyle='-')
    plt.title("Perda de Treinamento por Época (Fold 1)")
    plt.xlabel("Épocas")
    plt.ylabel("Perda de Treinamento (Loss)")
    plt.legend()
    plt.show()


## Criando a Cross Validation

In [79]:
from torch.utils.data import DataLoader
from sklearn.model_selection import TimeSeriesSplit

In [80]:

# Função para realizar a validação cruzada com TimeSeriesSplit e salvar resultados
def cross_validation_time_series(X, y, n_splits=5, batch_size=32, epochs=200):
    tscv = TimeSeriesSplit(n_splits=n_splits)  # Usando TimeSeriesSplit
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    fold_results = []  # Armazenar resultados de cada fold
    train_losses = []  # Armazenar as perdas de treinamento por época
    val_losses = []  # Armazenar as perdas de validação por fold
    all_predictions = []  # Armazenar todas as previsões feitas
    all_real_values = []  # Armazenar todos os valores reais

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        print(f"Fold {fold+1}/{n_splits}")
        
        # Dividir os dados em treino e validação
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Criar os datasets e dataloaders
        train_dataset = CustomDataset(X_train, y_train)
        val_dataset = CustomDataset(X_val, y_val)
        
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)  # Não embaralhar durante o treinamento
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # Não embaralhar durante a validação

        # Inicializar o modelo, critério e otimizador
        input_size = X.shape[2]  # Número de características (features)
        output_size = y.shape[1]  # Número de saídas
        modelo = LSTMModel(input_size=input_size, hidden_size=50, output_size=output_size).to(device)
        funcao_de_custo = nn.MSELoss()
        otimizador = torch.optim.Adam(modelo.parameters(), lr=1e-3)

        # Armazenar as perdas por época
        epoch_train_losses = []

        # Treinamento e validação
        for epoch in range(epochs):
            train_loss = train(modelo, train_dataloader, funcao_de_custo, otimizador, device)
            epoch_train_losses.append(train_loss)
            if epoch % 50 == 0:
                print(f"Epoch {epoch}/{epochs}, Train Loss: {train_loss:.4f}")
        
        # Armazenar a perda de validação e previsões
        val_loss = test(modelo, val_dataloader, funcao_de_custo, device)
        print(f"Validation Loss for fold {fold+1}: {val_loss:.4f}")
        fold_results.append(val_loss)
        val_losses.append(val_loss)

        # Armazenar previsões e valores reais para este fold
        modelo.eval()
        predicoes = []
        reais = []
        with torch.no_grad():
            for data, target in val_dataloader:
                data, target = data.float().to(device), target.float().to(device)
                pred = modelo(data)
                predicoes.append(pred.cpu().numpy())
                reais.append(target.cpu().numpy())
        
        predicoes = np.concatenate(predicoes, axis=0)
        reais = np.concatenate(reais, axis=0)
        
        all_predictions.append(predicoes)
        all_real_values.append(reais)
        train_losses.append(epoch_train_losses)
    
    # Média dos erros de validação
    avg_val_loss = np.mean(fold_results)
    print(f"Average Validation Loss across {n_splits} folds: {avg_val_loss:.4f}")
    
    # Agora chamamos a função plot_results para plotar os gráficos
    
    plot_results(fold_results, val_losses, train_losses, all_predictions, all_real_values)





# Executando

In [81]:
cross_validation_time_series(X_scaled, y_scaled, n_splits=5, batch_size=128, epochs=200)

Fold 1/5


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)